# 尝试实现手写Deepfm的简单功能

### 复现dataset.py文件内容

In [15]:
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

from model.DeepFM import DeepFM
from data.dataset import CriteoDataset

# 900000 items for training, 10000 items for valid, of all 1000000 items
# Num_train = 9000

实现下面代码之前需要train.txt,test.txt两个文件，

文件中每行是一个样本，列数指的是在每一个field中的取值(即不为0的数所在的index)

比如当前field为男女，那么取值只有0，1

若当前field是地区，那取值可能为0，1.....10000

In [16]:
# train_data = CriteoDataset('./data', train=True)
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os


continous_features = 13#13个连续变量

file_path = './data'
train_path = 'train.txt'
test_path = 'test.txt'

#读取特征和归属类别
df_train = pd.read_csv(os.path.join(file_path,'train.txt'))
feature = df_train.iloc[:,:-1].values
target = df_train.iloc[:,-1].values

In [17]:
#现在想要
#展示idx这一行的各个field数据以及target
idx = 20
dataI, targetI = feature[idx, :], target[idx]
print(dataI, targetI)

[   0    1   80    0 1848  287    1    4   46    0    1    0    4    1
    7    0 6497    1    1   64    1    1    1 1055    0  110    2    6
    0    3    5   12    3    0    0    1 2695    2  844] 0


In [18]:

Xi_coutinous = np.zeros_like(dataI[:continous_features])
Xi_categorial = dataI[continous_features:]
# 将连续特征和离散特征合并，并且增加一个维度
Xi = torch.from_numpy(np.concatenate((Xi_coutinous, Xi_categorial)).astype(np.int32)).unsqueeze(-1)
print(Xi)
print(Xi.size())

tensor([[   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   1],
        [   7],
        [   0],
        [6497],
        [   1],
        [   1],
        [  64],
        [   1],
        [   1],
        [   1],
        [1055],
        [   0],
        [ 110],
        [   2],
        [   6],
        [   0],
        [   3],
        [   5],
        [  12],
        [   3],
        [   0],
        [   0],
        [   1],
        [2695],
        [   2],
        [ 844]], dtype=torch.int32)
torch.Size([39, 1])


In [19]:
Xv_coutinous = dataI[:continous_features]#读取连续型变量的值
Xv_categorial = np.ones_like(dataI[continous_features:])#离散型变量的值都为1
Xv = torch.from_numpy(np.concatenate((Xv_coutinous, Xv_categorial)).astype(np.int32))
print(Xv)
print(Xv.size())

tensor([   0,    1,   80,    0, 1848,  287,    1,    4,   46,    0,    1,    0,
           4,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1], dtype=torch.int32)
torch.Size([39])


以上已完成数据处理，接下来构建Dataset

以上两个array构建原因有待填写：******

# 构建dataloader

In [20]:
# 900000 items for training, 10000 items for valid, of all 1000000 items
Num_train = 9000

#dataloader直接构建即可
train_data = CriteoDataset('./data', train=True)
loader_train = DataLoader(train_data, batch_size=16,
                          sampler=sampler.SubsetRandomSampler(range(Num_train)))
val_data = CriteoDataset('./data', train=True)
loader_val = DataLoader(val_data, batch_size=16,
                        sampler=sampler.SubsetRandomSampler(range(Num_train, 10000)))

print(loader_train)

下面两个程序很明显地展示了dataloeader的特点，它将一个batch的数据做成一组后会给整个数据组增加一个维度

In [21]:
for xi, xv, y in train_data:
    print(xi)
    print(xi.size())
    break

tensor([[   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   2],
        [  16],
        [  10],
        [  12],
        [   1],
        [   3],
        [2134],
        [   1],
        [   1],
        [ 786],
        [ 681],
        [  11],
        [ 661],
        [   1],
        [  41],
        [  12],
        [   2],
        [  16],
        [   1],
        [   2],
        [  12],
        [   0],
        [   2],
        [  17],
        [   2],
        [   9]], dtype=torch.int32)
torch.Size([39, 1])


In [22]:
# 迭代 loader_train，获取一批数据
for xi, xv, y in loader_train:
    print("xi:", xi)  # 打印 xi 的内容
    print(xi.size())
    cur_xi = xi
    break
    print("xv:", xv)  # 打印 xv 的内容
    print("y:", y)    # 打印 y 的内容
    break  # 只查看第一批数据，避免打印过多信息



xi: tensor([[[   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   1],
         [  46],
         [   0],
         [   5],
         [   1],
         [   1],
         [  55],
         [   1],
         [   1],
         [ 781],
         [  27],
         [   0],
         [  29],
         [   2],
         [  35],
         [   0],
         [   1],
         [  44],
         [   2],
         [   3],
         [   0],
         [   0],
         [   6],
         [   5],
         [   1],
         [   2]],

        [[   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   0],
         [   1],
         [  18],
         [1608],
         [2156],
         [   7],
         [   2],
        

# 构建DeepFM模型

In [23]:
feature_sizes = np.loadtxt('./data/feature_sizes.txt', delimiter=',')
feature_sizes = [int(x) for x in feature_sizes]
print(feature_sizes)
#存储了每一个field一共有几个参数

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 532, 533, 11289, 13822, 151, 13, 9005, 254, 4, 11557, 4191, 11560, 3037, 27, 5640, 12459, 11, 2790, 1391, 4, 12005, 10, 15, 9617, 52, 7599]


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from time import time

In [25]:
# fm_first_order_embeddings = nn.ModuleList([nn.Embedding(feature_size, 1) for feature_size in feature_sizes])

#两种方法都可以
fm_first_order_embeddings = nn.ModuleList()
for feature_size in feature_sizes:
    fm_first_order_embeddings.append(nn.Embedding(feature_size,1))

print(fm_first_order_embeddings)


ModuleList(
  (0-12): 13 x Embedding(1, 1)
  (13): Embedding(532, 1)
  (14): Embedding(533, 1)
  (15): Embedding(11289, 1)
  (16): Embedding(13822, 1)
  (17): Embedding(151, 1)
  (18): Embedding(13, 1)
  (19): Embedding(9005, 1)
  (20): Embedding(254, 1)
  (21): Embedding(4, 1)
  (22): Embedding(11557, 1)
  (23): Embedding(4191, 1)
  (24): Embedding(11560, 1)
  (25): Embedding(3037, 1)
  (26): Embedding(27, 1)
  (27): Embedding(5640, 1)
  (28): Embedding(12459, 1)
  (29): Embedding(11, 1)
  (30): Embedding(2790, 1)
  (31): Embedding(1391, 1)
  (32): Embedding(4, 1)
  (33): Embedding(12005, 1)
  (34): Embedding(10, 1)
  (35): Embedding(15, 1)
  (36): Embedding(9617, 1)
  (37): Embedding(52, 1)
  (38): Embedding(7599, 1)
)
